In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains


from datetime import datetime as dt
from datetime import timedelta
from time import sleep
import sys
import re
import os
import shutil
import os.path
import random
import urllib.parse as urlparse

import pandas as pd
import csv
import numpy as np

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment, Font, PatternFill

import warnings
warnings.filterwarnings('ignore')


from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.set_option('display.max_columns', None)

In [10]:
def scroll_until_found(driver, xpath, container_id="viewerContainer", step=400, max_scroll=50):
    container = driver.find_element(By.ID, container_id)
    for i in range(max_scroll):
        try:
            el = driver.find_element(By.XPATH, xpath)
            return el
        except:
            # Cuộn thêm xuống dưới
            driver.execute_script("arguments[0].scrollTop += arguments[1];", container, step)
            time.sleep(0.3)  # đợi text render
    return None

def process_adjacent_coordinates2(input_list):
    result = []
    i = 0
    
    while i < len(input_list) - 1:
        # Initialize group data for current row
        x1, y1, value1 = input_list[i]
        group_x_sum = x1
        group_values = [value1]
        count = 1
        i += 1
        
        # Start grouping consecutive items with the same y-coordinate
        while i < len(input_list) and input_list[i][1] == y1:
            group_x_sum += input_list[i][0]
            group_values.append(input_list[i][2])
            count += 1
            i += 1

        # Calculate average x and concatenate all values for the group
        avg_x = group_x_sum / count
        concatenated_values = " ".join(group_values)

        # Add the processed group to the result list
        result.append([avg_x, y1, concatenated_values])

    # If the last element wasn't processed (i.e., no pair for it), add it to the result
    if i < len(input_list):
        result.append(input_list[i])

    return result

def process_adjacent_coordinates(input_list):
    result = []
    
    i = 0
    while i < len(input_list) - 1:
        x1, y1, value1 = input_list[i]
        x2, y2, value2 = input_list[i + 1]

        if x1 == x2:
            avg_y = (y1 + y2) / 2
            concatenated_values = value1 + " " + value2
            result.append([x1, avg_y, concatenated_values])
            i += 2  # Skip the next element since it has already been processed
        else:
            result.append([x1, y1, value1])  # If no pair, keep the current item in result
            i += 1  # Move to the next element
    
    # If the last element wasn't processed (i.e., no pair for it), add it to the result
    if i < len(input_list):
        result.append(input_list[i])

    return result

def clean_lists(input_list):
    # Initialize an empty list to store the cleaned sublists
    cleaned_list = []

    # Regex pattern to match anything inside parentheses
    pattern = r'\((\d{1,2})\)'

    # Iterate through each sublist in the input list
    for sublist in input_list:
        if len(sublist) >= 5:
            cleaned_sublist = [item for item in sublist if not re.fullmatch(pattern, str(item))]

            cleaned_list.append(cleaned_sublist)

    return cleaned_list

In [3]:
# sys.path.append(
#     os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "..", ".."))
# )
sys.path.append(
    os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
)

In [26]:
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')

driver = webdriver.Chrome(options=options)

URL = "https://mof.gov.vn/bo-tai-chinh/tim-kiem?q=b%E1%BA%A3n%20tin%20n%E1%BB%A3%20c%C3%B4ng"
driver.get(URL)

In [28]:
elements = driver.find_elements(By.CSS_SELECTOR, "div.p-card a")

# lấy href
hrefs = [el.get_attribute("href") for el in elements]
print(hrefs)

['https://mof.gov.vn/bo-tai-chinh/ban-tin-no-cong/mofucm304557', 'https://mof.gov.vn/bo-tai-chinh/ban-tin-no-cong/mofucm288663', 'https://mof.gov.vn/bo-tai-chinh/ban-tin-no-cong/mofucm270034', 'https://mof.gov.vn/bo-tai-chinh/ban-tin-no-cong/mofucm241517', 'https://mof.gov.vn/bo-tai-chinh/ban-tin-no-cong/mofucm228895', 'https://mof.gov.vn/bo-tai-chinh/ban-tin-no-cong/mofucm216051', 'https://mof.gov.vn/bo-tai-chinh/ban-tin-no-cong/mofucm216049', 'https://mof.gov.vn/bo-tai-chinh/quan-ly-no-kinh-te-doi-ngoai/mofucm143311', 'https://mof.gov.vn/bo-tai-chinh/quan-ly-no-kinh-te-doi-ngoai/btc313164']


In [29]:
driver.get(hrefs[0])

In [30]:
table_element = driver.find_element(By.CLASS_NAME, 'page')
table_html = table_element.get_attribute('outerHTML')

In [31]:
target_xpath = "//span[contains(text(),'Mẫu biểu công bố thông tin số 4.01')]"
target = scroll_until_found(driver, target_xpath)

if target:
    driver.execute_script("arguments[0].scrollIntoView();", target)
    print("Đã tìm thấy:", target.text)
else:
    print("Không tìm thấy span cần tìm")
    
page_div = target.find_element(By.XPATH, "./ancestor::div[contains(@class,'page')]")
page_number = page_div.get_attribute("data-page-number")
print("Target nằm ở trang:", page_number)

page = driver.find_element(By.XPATH, f"//div[@class='page' and @data-page-number='{page_number}']")
spans = page.find_elements(By.CSS_SELECTOR, ".textLayer span")
rows = []
for s in spans:
    text = s.text.strip()

    if not text:
        continue
    style = s.get_attribute("style")
    left_val = None
    top_val = None
    for part in style.split(";"):
        if "left:" in part:
            left_val = float(part.replace("left:", "").replace("px", "").strip())
        if "top:" in part:
            top_val = float(part.replace("top:", "").replace("px", "").strip())
    if text != 'Trong đó:':
        rows.append((top_val, left_val, text))
        
lines = {}
for top, left, txt in process_adjacent_coordinates2(rows):
    key = round(top, 0)
    lines.setdefault(key, []).append(txt)
    
sorted_lines = [lines[k] for k in sorted(lines.keys())]
table_lines = clean_lists(sorted_lines)

Đã tìm thấy: Mẫu biểu công bố thông tin số 4.01
Target nằm ở trang: 12


In [32]:
pd.DataFrame(table_lines[1:], columns = table_lines[0])

,Chỉ tiêu,2018,2019,2020,2021,2022 (P)
0,1.Nợ công so với tổng sản phẩm quốc dân(GDP) (%),"58,3","55,0","55,9","42,7","37,4"
1,a. Nợ Chính phủ so với tổng sản phẩm quốc dân ...,"49,9","48,0","49,9","38,7","34,2"
2,b. Nợ Chính phủ bảo lãnh so với tổng sản phẩm ...,"7,9","6,7","5,8","3,8","3,1"
3,c. Nợ Chính quyền địa phương so với tổng sản p...,"0,9","0,7","0,7","0,6","0,6"
4,2. Nợ nước ngoài của quốc gia so với tổng sản ...,"46,0","47,1","47,9","38,1","36,1"
5,3. Nghĩa vụ trả nợ nước ngoài của quốc gia so ...,"7,0","5,9","5,7","6,2","6,9"
6,4. Nghĩa vụ trả nợ của Chính phủ so với thu ng...,"17,1","17,4","21,2","21,5","15,7"


In [13]:
def modify_layered_table(df: pd.DataFrame) -> pd.DataFrame:
    """
    Input df columns (example):
      ['Chỉ tiêu','2018','2019','2020','2021','2022 (P)']
    Output columns:
      ['ID','Năm','Chỉ tiêu L1','Chỉ tiêu L2','Tỉ lệ thực tế','Tỉ lệ dự tính']
    """
    df = df.copy()

    # 1) Identify year columns (e.g., '2021', '2022 (P)')
    year_cols = [c for c in df.columns if re.match(r'^\s*20\d{2}', str(c))]
    pred_cols  = [c for c in year_cols if re.search(r'\(P\)', str(c))]           # predicted
    actual_cols = [c for c in year_cols if c not in pred_cols]                   # actual

    # 2) Normalize numbers (supports '37,4' → 37.4)
    for c in year_cols:
        df[c] = (df[c].astype(str)
                      .str.replace(r'[^\d,.\-]', '', regex=True)
                      .str.replace(',', '.', regex=False)
                      .replace({'', np.nan})
                      .replace({'nan': np.nan})
                      .astype(float))

    # 3) Walk the table and emit tidy rows
    out = []
    current_l1, current_l2, last_level = None, None, None
    re_l1 = re.compile(r'^\s*\d+\.\s*(.*)$')
    re_l2 = re.compile(r'^\s*[a-z]\.\s*(.*)$', re.I)

    for _, r in df.iterrows():
        label = str(r['Chỉ tiêu']).strip()
        m1, m2 = re_l1.match(label), re_l2.match(label)

        if m1:                               # Layer 1 row
            current_l1 = m1.group(1)
            current_l2 = None
            last_level = 'l1'
        elif m2:                             # Layer 2 row
            current_l2 = m2.group(1)
            last_level = 'l2'
        else:
            # Continuation line (rare): append to last seen title
            if last_level == 'l2' and current_l2:
                current_l2 = f"{current_l2} {label}"
            elif current_l1:
                current_l1 = f"{current_l1} {label}"

        # Emit rows for all year cells present
        for c in actual_cols:
            val = r.get(c, pd.NA)
            if pd.notna(val):
                out.append({
                    'Năm': int(re.findall(r'20\d{2}', str(c))[0]),
                    'Chỉ tiêu L1': current_l1,
                    'Chỉ tiêu L2': current_l2 or '',
                    'Tỉ lệ thực tế': float(val),
                    'Tỉ lệ dự tính': np.nan
                })
        for c in pred_cols:
            val = r.get(c, pd.NA)
            if pd.notna(val):
                out.append({
                    'Năm': int(re.findall(r'20\d{2}', str(c))[0]),
                    'Chỉ tiêu L1': current_l1,
                    'Chỉ tiêu L2': current_l2 or '',
                    'Tỉ lệ thực tế': np.nan,
                    'Tỉ lệ dự tính': float(val)
                })

    out_df = pd.DataFrame(out)

    # Order & add ID
    out_df = (out_df
              .sort_values([ 'Năm', 'Chỉ tiêu L1', 'Chỉ tiêu L2',], ascending=[False, True, True])
              .reset_index(drop=True))
    out_df.insert(0, 'ID', range(1, len(out_df) + 1))
    return out_df

In [14]:
modify_layered_table(pd.DataFrame(table_lines[1:], columns = table_lines[0]))

,ID,Năm,Chỉ tiêu L1,Chỉ tiêu L2,Tỉ lệ thực tế,Tỉ lệ dự tính
0,1,2022,Nghĩa vụ trả nợ của Chính phủ so với thu ngân ...,,NaN,15.7
1,2,2022,Nghĩa vụ trả nợ nước ngoài của quốc gia so với...,,NaN,6.9
2,3,2022,Nợ công so với tổng sản phẩm quốc dân(GDP) (%),,NaN,37.4
3,4,2022,Nợ công so với tổng sản phẩm quốc dân(GDP) (%),Nợ Chính phủ bảo lãnh so với tổng sản phẩm quố...,NaN,3.1
4,5,2022,Nợ công so với tổng sản phẩm quốc dân(GDP) (%),Nợ Chính phủ so với tổng sản phẩm quốc dân (GD...,NaN,34.2
5,6,2022,Nợ công so với tổng sản phẩm quốc dân(GDP) (%),Nợ Chính quyền địa phương so với tổng sản phẩn...,NaN,0.6
6,7,2022,Nợ nước ngoài của quốc gia so với tổng sản phẩ...,,NaN,36.1
7,8,2021,Nghĩa vụ trả nợ của Chính phủ so với thu ngân ...,,21.5,NaN
8,9,2021,Nghĩa vụ trả nợ nước ngoài của quốc gia so với...,,6.2,NaN
9,10,2021,Nợ công so với tổng sản phẩm quốc dân(GDP) (%),,42.7,NaN


In [33]:
my_dict = {}
list_tables = ['Mẫu biểu công bố thông tin số 4.02',
              'Mẫu biểu công khai thông tin số 4.03',
              'Mẫu biểu công bố thông tin số 4.04',
              'Mẫu biểu công bố thông tin số 4.05',
              'Mẫu biểu công bố thông tin số 4.06'
              ]

In [34]:
for table in list_tables:
    print(table)
    try:
        target_xpath = f"//span[contains(text(), '{table}')]"
        target = scroll_until_found(driver, target_xpath)

        if target:
            driver.execute_script("arguments[0].scrollIntoView();", target)
            print("Đã tìm thấy:", target.text)
        else:
            print("Không tìm thấy span cần tìm")

        page_div = target.find_element(By.XPATH, "./ancestor::div[contains(@class,'page')]")

        # lấy số trang
        page_number = page_div.get_attribute("data-page-number")
        print("Target nằm ở trang:", page_number)

        page = driver.find_element(By.XPATH, f"//div[@class='page' and @data-page-number='{page_number}']")

        # 2. Lấy toàn bộ span trong textLayer của page 12
        spans = page.find_elements(By.CSS_SELECTOR, ".textLayer span")

        rows = []
        for s in spans:
            text = s.text.strip()
            
            if not text:
                continue
            style = s.get_attribute("style")
            left_val = None
            top_val = None
            for part in style.split(";"):
                if "left:" in part:
                    left_val = float(part.replace("left:", "").replace("px", "").strip())
                if "top:" in part:
                    top_val = float(part.replace("top:", "").replace("px", "").strip())
            if text != 'Trong đó:':
                rows.append((top_val, left_val, text))

        lines = {}
        for top, left, txt in process_adjacent_coordinates(rows):
            key = round(left, 0)
            lines.setdefault(key, []).append(txt)

        # 4. Sắp xếp theo top
        sorted_lines = [lines[k] for k in sorted(lines.keys())]
    
        table_lines = clean_lists(sorted_lines)
        table_lines[0] = [item for item in table_lines[0] for _ in range(2)]
        table_lines[0].insert(0, 'Year')
        table_lines[1].insert(0, 'Currency')
        df = pd.DataFrame(table_lines)
        my_dict[table] = df
        
    except:
        print('Error')
    

Mẫu biểu công bố thông tin số 4.02
Đã tìm thấy: Mẫu biểu công bố thông tin số 4.02
Target nằm ở trang: 13
Mẫu biểu công khai thông tin số 4.03
Đã tìm thấy: Mẫu biểu công khai thông tin số 4.03
Target nằm ở trang: 14
Mẫu biểu công bố thông tin số 4.04
Đã tìm thấy: Mẫu biểu công bố thông tin số 4.04 
Target nằm ở trang: 15
Mẫu biểu công bố thông tin số 4.05
Đã tìm thấy: Mẫu biểu công bố thông tin số 4.05
Target nằm ở trang: 16
Mẫu biểu công bố thông tin số 4.06
Đã tìm thấy: Mẫu biểu công bố thông tin số 4.06
Target nằm ở trang: 17


In [36]:
my_dict[list_tables[4]]

,0,1,2,3,4,5,6,7,8,9,10
0,Year,2019,2019,2020,2020,2021,2021,2022 (P),2022 (P),6/2023 (P),6/2023 (P)
1,Currency,USD,VND,USD,VND,USD,VND,USD,VND,USD,VND
2,DƯ NỢ (1),"122.779,59","2.841.488,16","130.118,98","3.016.287,89","139.529,60","3.226.761,46","144.857,57","3.430.372,18","144.195,14","3.411.656,90"
3,Nợ nước ngoài của Chính phủ,"47.733,63","1.104.699,34","49.008,24","1.136.059,94","46.552,13","1.076.564,56","41.171,97","974.993,41","41.570,46","983.556,99"
4,Nợ nước ngoài của doanh nghiệp,"75.045,97","1.736.788,82","81.110,74","1.880.227,95","92.977,47","2.150.196,90","103.685,60","2.455.378,77","102.624,68","2.428.099,91"
5,RÚT VỐN VAY TRONG KỲ (2),"93.331,96","2.148.139,22","115.365,41","2.677.502,74","140.918,89","3.263.217,02","159.078,78","3.695.736,59","57.580,91","1.361.369,88"
6,Vay nước ngoài của Chính phủ,"2.285,81","52.515,11","2.230,40","51.658,29","1.933,36","44.688,24","1.954,23","45.524,27","659,30","16.369,15"
7,Vay nước ngoài của doanh nghiệp,"91.046,15","2.095.624,11","113.135,01","2.625.844,45","138.985,53","3.218.528,78","157.124,55","3.650.212,32","56.921,61","1.345.000,73"
8,TỔNG TRẢ NỢ TRONG KỲ (2),"86.480,49","1.990.480,35","112.629,84","2.614.081,45","131.617,00","3.047.870,30","152.054,06","3.532.430,73","61.563,34","1.454.685,65"
9,Nợ nước ngoài của Chính phủ,"2.556,14","58.819,63","3.508,12","81.384,86","3.156,35","73.084,91","3.412,18","79.282,16","1.702,74","40.239,50"
